# SPD

Replicate the design of [Smaldino et al. (2013)](https://www.journals.uchicago.edu/doi/10.1086/669615).

## Imports

In [ ]:
# model
from mesa_fork import Model, Agent
from mesa_fork.time import RandomActivation
from mesa_fork.space import SingleGrid
from mesa_fork.datacollection import DataCollector
from enum import Enum

# visualization
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import holoviews as hv
%load_ext holoviews.ipython
import seaborn as sns
sns.set_theme(style="darkgrid")

# parameter sweep
from mesa_fork.batchrunner import BatchRunner, BatchRunnerMP

## Setup model

In [ ]:
# PD game

class Action(Enum):
    COOPERATE = 1
    DEFECT    = 2

    
def pd_game(alice, bob, R=3, T=5, S=-1, P=0):
    
    a = alice.play()
    b = bob.play()

    if a == Action.COOPERATE:
        if b == Action.COOPERATE:
            alice.energy += R
            bob.energy   += R
        else:
            alice.energy += S
            bob.energy   += T
    else:
        if b == Action.COOPERATE:
            alice.energy += T
            bob.energy   += S
        else:
            alice.energy += P
            bob.energy   += P

    alice.energy = min(alice.energy, alice.max_energy)
    bob.energy = min(bob.energy, bob.max_energy)
        

In [ ]:
class SmaldinoAgent(Agent):
    
    def __init__(self, 
                 model,
                 energy,
                 max_energy,
                 cooperator=False):
        
        super().__init__(model.next_id(), model)
        
        self.energy = energy
        self.max_energy = max_energy
        self.cooperator = cooperator
        
        self.played = False
        self.newborn = False
        
        
    def play(self):
        if self.cooperator:
            return Action.COOPERATE
        else:
            return Action.DEFECT
        
        
    def step(self):
        
        # don't step if created this turn
        if not self.newborn:
        
            neighbors = self.model.grid.get_neighbors(self.pos, moore=True)
            opponents = list(filter(
                lambda a: not a.played,
                neighbors))

            # find opponent
            if opponents:
                opponent = self.random.choice(opponents)

                # play pd game
                if not self.played:

                    pd_game(self, opponent, self.model.R, self.model.T, self.model.S, self.model.P)
                    self.played = True
                    opponent.played = True

                # reproduce
                max_population = (self.model.grid.width * self.model.grid.height) / 2
                if (    self.cooperator and self.model.cooperator_count < max_population) or \
                   (not self.cooperator and self.model.defector_count   < max_population):

                    neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True)
                    unoccupied = list(filter(
                        lambda c: self.model.grid.is_cell_empty(c), 
                        neighborhood))

                    if self.energy > (self.model.energy_to_reproduce * 2) and unoccupied:

                        cell = self.random.choice(unoccupied)

                        offspring = SmaldinoAgent(self.model,
                                                  self.model.energy_to_reproduce,
                                                  self.max_energy,
                                                  self.cooperator)
                        offspring.newborn = True
                        offspring.played = True

                        self.model.grid.position_agent(offspring, cell[0], cell[1])
                        self.model.schedule.add(offspring)
                        
                        if self.cooperator:
                            self.model.cooperator_count += 1
                        else:
                            self.model.defector_count += 1

                        self.energy -= self.model.energy_to_reproduce

            elif not self.played:
                # attempt movement
                neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True)
                unoccupied = list(filter(
                    lambda c: self.model.grid.is_cell_empty(c), 
                    neighborhood))

                if unoccupied:
                    cell = self.random.choice(unoccupied)
                    self.model.grid.move_agent(self, cell)

            # energy deduction (cost of living)
            self.energy -= self.model.living_cost
            if self.energy <= 0:
                # die
                self.model.grid.remove_agent(self)
                self.model.schedule.remove(self)
                return
            
        # update values for DataCollector
        self.model.agent_count += 1

        if self.cooperator:
            self.model.cooperator_count += 1
        else:
            self.model.defector_count += 1


In [ ]:
class SPDModel(Model):
    
    def __init__(self,
                 R=3, T=5, S=-1, P=0,
                 starting_agent_count=10,
                 starting_energies=range(1,51),
                 max_energy=150,
                 energy_to_reproduce=50,
                 living_cost=1,
                 grid_size=10,
                 wrap=True):
        """
        Smaldino's spatial prisonner's dilemma model
        
        Args:
            R, T, S, P:            PD payoffs
            starting_agent_count:  starting number of agents
            starting_energies:     list of possible starting energies for agents (picked at random)
            max_energy:            maximal energy an agent can hold
            energy_to_reproduce:   energy required to reproduce
            living_cost:           energy deducted at the end of each step
            grid_size:             size length of square grid to use
            wrap:                  whether to wrap grid (torus bounds)
        """
        
        super().__init__()
        self.schedule = RandomActivation(self)
        self.grid = SingleGrid(grid_size, grid_size, torus=wrap)
        
        self.R = R
        self.T = T
        self.S = S
        self.P = P
        self.energy_to_reproduce = energy_to_reproduce
        self.living_cost = living_cost
        
        # Setup agents
        self.cooperator_count = 0
        self.defector_count = 0

        for i in range(starting_agent_count):
            energy = self.random.choice(starting_energies)
            cooperator = i%2 == 0
            
            if cooperator:
                self.cooperator_count += 1
            else:
                self.defector_count += 1
            
            agent = SmaldinoAgent(self, 
                                  energy,
                                  max_energy,
                                  cooperator=cooperator)
        
            cell = self.random.choice(list(self.grid.empties))

            self.grid.position_agent(agent, cell[0], cell[1])
            self.schedule.add(agent)
        
        self.agent_count = starting_agent_count
        
        # Init model
        self.running = True
        
        self.datacollector = DataCollector(
            {
                "agent_count": "agent_count",
                "cooperator_count": "cooperator_count",
                "defector_count": "defector_count",
            },
        )
        self.datacollector.collect(self)
        
        
    def step(self):
        
        # setup for step
        self.agent_count = 0
        self.cooperator_count = 0
        self.defector_count = 0

        for a in self.schedule.agents:
            a.played = False
            a.newborn = False
    
        # step
        self.schedule.step()
        self.datacollector.collect(self)
        
        # stop the model if no agents are alive
        if self.cooperator_count == 0 or self.defector_count == 0:
            self.running = False


## Run model

In [ ]:
spd = SPDModel(R=3, T=5, S=-1, P=0,
               starting_agent_count=64,
               starting_energies=range(1,50),
               max_energy=150,
               energy_to_reproduce=50,
               living_cost=1,
               grid_size=20,
               wrap=True)

In [ ]:
i = 0
while spd.running and i < 1000:
    spd.step()
    i += 1

### Check results

In [ ]:
results = spd.datacollector.get_model_vars_dataframe()

sns.lineplot(data=results[['cooperator_count', 'defector_count']])

### Render visualization

In [ ]:
def value(cell):
    if cell is None:
        return 0
    elif isinstance(cell, Agent):
        if cell.cooperator:
            return 2
        else:
            return 10
    else:
        raise Exception("Unidentified cell: {}".format(cell))
        
hmap = hv.HoloMap(kdims='step')
i = 0
while spd.running and i < 100:
    spd.step()
    data = np.array([[value(c) for c in row] for row in spd.grid.grid])
    hmap[i] = hv.Image(data, vdims=[hv.Dimension('State', range=(0,10))])
    i += 1
hmap

## Paramater sweep

In [ ]:
variable_parameters = {
    "S": np.linspace(-2.5, 0, 10),
}
fixed_parameters = {
    "R": 3,
    "T": 5,
    "P": 0,
    "starting_agent_count":  64,
    "starting_energies":     range(1,50),
    "max_energy":            150,
    "energy_to_reproduce":   50,
    "living_cost":           1,
    "grid_size":             20,
    "wrap":                  True,
}

iterations = 1
max_steps = 1000

param_run = BatchRunnerMP(SPDModel,
                          nr_processes=None,  # detect automatically
                          variable_parameters=variable_parameters,
                          fixed_parameters=fixed_parameters,
                          iterations=iterations,
                          max_steps=max_steps,
                          model_reporters={
                              "agent_count": lambda m: m.agent_count,
                              "cooperator_count": lambda m: m.cooperator_count,
                              "defector_count": lambda m: m.defector_count,
                          })

param_run.run_all()

In [ ]:
run_data = param_run.get_model_vars_dataframe()
run_data['cooperator_frequency'] = (run_data['cooperator_count'] / run_data['agent_count'])
run_data = run_data.dropna()
run_data.head()

In [ ]:
sns.boxplot(x="S", y="cooperator_frequency", data=run_data)